![title](birds-nest-4-eggs.png)

###### image:https://www.publicdomainpictures.net/nl/view-image.php?image=61631&picture=vogels-nest-4-eieren

#   The effect of precipitation between March and July on the breeding of birds in the Netherlands 1990-2020 in changes compared to the year before.

Sources:

Dutch bird breeding per season as percentage compared to indexyear (all years and all birds manually selected):
https://opendata.cbs.nl/statline/#/CBS/nl/dataset/84498NED/table?ts=1673294982549

Monthly sum of precipitation in 0.1 mm (ROYAL NETHERLANDS METEOROLOGICAL INSTITUTE):
###### De Kooy: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_235_rh24.txt
###### De Bilt: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_rh24.txt
###### Leeuwarden: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_270_rh24.txt
###### Eelde: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_280_rh24.txt
###### Twenthe: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_290_rh24.txt
###### Schiphol: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_240_rh24.txt
###### Rotterdam: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_344_rh24.txt
###### Vlissingen: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_310_rh24.txt
###### Eindhoven: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_370_rh24.txt
###### Maastricht/Beek: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_380_rh24.txt

Additional info:
###### *Coordinates for the stations: http://climexp.knmi.nl/KNMIData/list_dx.txt
###### *geojson for provinces: https://www.webuildinternet.com/articles/2015-07-19-geojson-data-of-the-netherlands/provinces.geojson






In [ ]:
#Importing all necessary libraries
import pandas as pd
import functions_final_assignment as fn
import numpy as np

#Loading in the files with a yaml config file
config = fn.yaml_config()

#Loading the data into a dataframe
precipitation_df = fn.load_concat_df(config["precipitation"])

birds_df = pd.read_excel(io=config["breedingbirds"],
           sheet_name="Provinciale trends 1990-2020",skiprows=2)

#loading a dataframe with the centerpoints of the provinces
geo_df = fn.read_geojson("DATA/provinces.geojson")

## Weather Data preparation

In [ ]:

#getting the middle points of the provinces.
geo_df["middle_point"] = [fn.get_centerpoint(data)
                         for data
                         in geo_df["geometry.coordinates"]]

geo_df = geo_df[["properties.name","middle_point"]]


Every centroid except for that of Noord-brabant is calculated correctly,
this could be due to some encapsulated regions. 

In [ ]:
#took the geographic mid point for brabant from google maps (its a monument)

geo_df["middle_point"].loc[6] = [51.562212646388495, 5.185266108595458]
geo_df.head(7)

The precipitation data starts well before 1990, as we only need the data between 1990 and 2020 we can get rid of most data.
After that the amount of missing values may be calculated.

In [ ]:
#select only the rows with the values in the YYYY column between 1990 and 2020
precipitation_df = precipitation_df[precipitation_df.YYYY.between(1990,2020)]
#Show the unique values for YYYY to see if the YYYY filtering is done correctly
print(f'{precipitation_df.YYYY.unique()}')


In [ ]:
#Calculated the amount of missing values
print(f'The amount of missing values are:\n {precipitation_df.isnull().sum()}')


In [ ]:
#convert all values to integers:
precipitation_df = precipitation_df.astype(int)

#Adding the location at which the station is found to the dataframe
stn_dict = fn.make_stn_dict(config["stn_coord"])
precipitation_df["COORD"] = [stn_dict[str(s)] for s in precipitation_df.STN]
precipitation_df.head()

In [ ]:
#As we want to see the sum for precipitation between march and july
#A column will be added
precipitation_df["MAR-JUL"] = precipitation_df.iloc[:,4:9].T.sum()
precipitation_df = precipitation_df[["STN","YYYY","COORD","MAR-JUL"]]
precipitation_df.head()

In [ ]:
#plot to see if the rain in March and July is normally distributed
#Also to see what the mean and deviation is. 
fn.hist_robust_dist(precipitation_df["MAR-JUL"])

There is some right skewedness as wheather extremes are not uncommon. But the data looks about distributed normally
with a Q-Q plot we can check this. 

In [ ]:
fn.DS_Q_Q_Plot(precipitation_df["MAR-JUL"])


In [ ]:
stn_df = precipitation_df[["STN","COORD"]] # take the stn and coordinate columns
stn_df = stn_df.drop_duplicates(subset=["STN"])

#calculate the distance from the middle of the provinces to the stations
for c, column in enumerate(geo_df["properties.name"]):
    stn_df[column] = [fn.calc_point_dist(geo_df.iloc[c,1],x) 
                                for x  
                                in stn_df["COORD"]]

#------Giving weight to the stations for different provinces:
# Calculates the distance (sum of coordinates) between the weatherstations
#  and the middle point of the province.
# Now the ratios will be calculated (province/sum) and the distances will be
#overwritten.
for column in stn_df.columns[2:]:
    #first the sum of the distances is calculated for a province
    col_sum = stn_df[column].sum()
    #The squared inverse is calculated 1/(distance to a station/sum of the distances) 
    stn_df[column] = [ (1/(i/col_sum)) for i in stn_df[column]]
    #now the new sum is calculated
    col_sum = stn_df[column].sum()
    # now the final ratios/weights are calculated
    stn_df[column] = [ (i/col_sum) for i in stn_df[column]]

stn_df.head(12)

In [ ]:
precipitation_df = precipitation_df.set_index(['YYYY','STN']).sort_index()

precipitation_df = precipitation_df.drop(columns="COORD")

for i in stn_df.columns[2:]:
    precipitation_df[i] = [precipitation_df["MAR-JUL"].iloc[c]
                           *
                           stn_df[i].iloc[fn.get_position(c)]
                           for 
                           c, year in enumerate(precipitation_df.index)]

precipitation_df = precipitation_df.reset_index()
precipitation_df = precipitation_df.drop(columns=["STN","MAR-JUL"])

years = precipitation_df.YYYY.unique()

def get_sum_precipitation():
    dict_sum = {"Years":years}
    for province in precipitation_df.columns[1:]:
        sums = []
        for year in years:
            sums.append(sum(precipitation_df[province][precipitation_df["YYYY"] == year]))
        dict_sum.update({province:sums})
    return dict_sum

province_prec_df = pd.DataFrame(get_sum_precipitation())

#Sorry Fenna:
province_prec_df = province_prec_df.rename(columns={"Friesland (Fryslân)":"Friesland"})

province_prec_df

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as pw

prec_province_df = province_prec_df.copy()

def plot(Province):
    plt.bar(x=prec_province_df.Years,height=prec_province_df[Province])
    plt.xlabel("Years");plt.ylabel("Precipitation (in 0.1mm)")
    plt.title(f'Precipitation in {Province}');plt.xticks(years,rotation=90)

prec_province = pw.interactive(plot,Province=prec_province_df.columns[1:])

prec_province

As seen in the interactive plots, 1996 is an exceptionally dry year. 2007 in contrast looks way wetter than average. 

## Birds Dataframe preparation

In [ ]:
#and now prepare the data in the birds df:

birds_df.head()

Some descriptives in the first three and in the last four columns we do not need. 

In [ ]:
#drop the first three and the last four columns
birds_df = birds_df.iloc[:,3:-4]

In [ ]:
#change the values to delta precentage change
#replace the data with percentage change, with 1990 as 0. np.inf and -np.inf are converted to zero.
birds_df = birds_df.T
birds_df.iloc[2:,:] = birds_df.iloc[2:,:].pct_change().replace({np.inf:np.NaN,-np.inf:np.NaN})
birds_df = birds_df.T.rename(columns={"Provincie":"Province"})
birds_df = birds_df.rename(columns={x:int(x) for x in birds_df.columns[2:]})
birds_df = birds_df.set_index("English name")


In [ ]:
birds_df = birds_df.reset_index().set_index(["English name","Province"])
birds_df = birds_df.T


#list with all birds we need
list_bird = [bird[0] for bird in birds_df.columns.unique()]
list_birds = list(set(list_bird))
#list with the provinces we need
list_provinces = [bird[1] for bird in birds_df.columns.unique()]
list_provinces = list(set(list_provinces))


#First create a dictionary 
values = [province_prec_df[i] for i in list_provinces]
values = [value for i in values for value in i]

dict_for_df = {"Precipitation":values}


for bird in list_birds:
    bird_values = []
    for province in list_provinces:
        try: 
            b_val = [value for value in birds_df[bird,province]]
            bird_values.append(b_val)
        except:
            n_val = [np.NaN for i in range(31)]
            bird_values.append(n_val)
    bird_values = [value for nested in bird_values for value in nested]
    dict_for_df.update({bird:bird_values})

filtered_df = pd.DataFrame(dict_for_df)

# Some birds are very region specific, so we keep only the most common birds
filtered_df = filtered_df.dropna(axis=1,thresh=300)

filtered_df



In [ ]:
#correlate different precipitation conditions with the birds

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


ranges = [int(i) for i in np.linspace(1000,5000,5)]

#Function to create a dataframe with correlations based on precipitation
def make_prec_corr(dataframe):
    corr = pd.DataFrame()
    for i in range(4):
        precipitation = dataframe.copy()
        precipitation = precipitation[precipitation["Precipitation"].between(ranges[i],ranges[i+1])]
        precipitation = precipitation.rename(columns={"Precipitation":f"{ranges[i]}-{ranges[i+1]}"})
        precipitation = precipitation.corrwith(precipitation[f"{ranges[i]}-{ranges[i+1]}"])
        corr[f"{ranges[i]}-{ranges[i+1]}"] = precipitation
    return corr.iloc[1:]



corr = make_prec_corr(filtered_df)

#filter based on 

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(5, 8))

#Generate a custom diverging colormap
cmap = sns.diverging_palette(20, 200, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, center=0, cbar_kws={"shrink": .4})
ax.set_xlabel("Precipitation (*0.1mm)")
ax.set_title("The effect of precipitation on breeding birds")
